In [ ]:
# RESTful Endpoint
from dataclasses import dataclass
from flask import Response, Request, Flask, jsonify, request
from typing import List, Dict, Literal
from typing import Union, Tuple, Dict
from flask import jsonify, request
app = Flask(__name__)

# income model class
@dataclass
class Income:
    id: int
    description: str
    amount: float

    def update(self, new_income):
        if 'description' in new_income: self.description = new_income['description']
        if 'amount' in new_income: self.amount = new_income['amount']
    
    def __repr__(self) -> dict:
        return {
            'id': self.id,
            'description': self.description,
            'amount': self.amount
        }

incomes: list[Income] = [
    Income(1, 'salary', 5000),
]

@app.route('/', methods=['GET'])
def get_incomes() -> Union[str, Tuple[Dict[str, str], int]]:
    if request.method == 'GET':
        return jsonify(incomes).data.decode('utf-8'), 200
    
    return {'error': 'method not allowed'}, 405

@app.route('/incomes', methods=['POST'])
def add_incomes() -> tuple[dict[str, str], int]:
    if request.method == 'POST':
        if not request.is_json: return {'error': 'not a json'}, 400        
        if request.get_json() is None: return {'error': 'invalid json'}, 400
        
        new_incomes_list: list[Income] = []

        largest_id = max((income.id for income in incomes), default=0)
        if isinstance(request.get_json(), list):
            for data in request.get_json():
                if 'description' not in data or 'amount' not in data:
                    return {'error': 'invalid json'}, 400
                largest_id += 1
                income: Income = Income(largest_id, data['description'], data['amount'])
                new_incomes_list.append(income)                
        else:
            if 'description' not in request.get_json() or 'amount' not in request.get_json():
                return {'error': 'invalid json'}, 400
            largest_id += 1
            income: Income = Income(largest_id, request.get_json()['description'], request.get_json()['amount'])
            new_incomes_list.append(income)

        incomes.extend(new_incomes_list)
        return {'success': "Records created"}, 201

    return {'error': 'method not allowed'}, 405

@app.route('/incomes/<int:income_id>', methods=['PUT','DELETE','GET'])
def incomes_api(income_id: int) -> tuple[dict[str, str], int]:
    if request.method == 'DELETE': 
        if __delete_income(income_id): return {'success': "Record deleted"}, 200
        return {'error': 'not found'}, 404
    
    if request.method == 'PUT':
        if not request.is_json: return {'error': 'not a json'}, 400 
        if request.get_json() is None: return {'error': 'invalid json'}, 400
        if __update_income(income_id, request.get_json()): return {'success': "Record updated"}, 200
        return {'error': 'not found'}, 404

    if request.method == 'GET':
        income = __get_income(income_id)

        if income is None: return {'error': 'not found'}, 404
        return jsonify(income).data.decode('utf-8'), 200

    return {'error': 'method not allowed'}, 405

def __delete_income(income_id: int) -> bool:
    for index, income in enumerate(incomes):
        if income.id == income_id:
            del incomes[index]
            return True
    return False

def __update_income(income_id: int, new_income) -> bool:
    for index, income in enumerate(incomes):
        if income.id == income_id:
            incomes[index].update(new_income) #type: ignore
            return True
    return False

def __get_income(income_id: int) -> Union[Income, None]:
    for income in incomes:
        if income.id == income_id: return income
    return None

if __name__ == '__main__':
    app.run()